# Test Initialization on Neural Network

## Table of Contents

* [1. Build a L-layers Neural Network ](#chapter1)
    * [1.1 Initialize parameters](#section_1_1)
    * [1.2 Forward propagation](#section_1_2)
    * [1.3 Cost function](#section_1_3)
    * [1.4 Backward Propagation](#section_1_4)
    * [1.5 Update parameters](#section_1_5) 
    * [1.6 Predict](#section_1_6) 
    * [1.7 Model](#section_1_7)
* [2. Example on Dataset 1](#chapter2)
    * [2.1 Load the Dataset](#section_2_1)
    * [2.2 Display the Data](#section_2_2)
    * [2.3 Flatten the data](#section_2)
    * [2.4 Normalize the data](#section_2_4)
* [3. Example on Dataset 2](#chapter3)
    

In [4]:
# Packages
import copy
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss


%matplotlib inline

# 1. Build a L-layer Neural Network <a class="anchor" id="chapter1"></a>

## 1.1 initialization parameters  <a class="anchor" id="section_1_1"></a>

In [17]:
def initialize_parameters(layers_dims,n_input,n_output,type_init="random"):
    """
    Compute the initialization of the parameters in our Neural Network

    - Arguments:
    layers_dims: array containing the dimension of the hidden layers
    n_input: numbers of features in the input layer
    n_ouput: numbers of nodes in the output layer
    type_init: "zeros","random","he" type of initialization

    - Return:
    parameters: dictionnary containing of the parameters of our neural network
    """

    # init
    parameters = {}

    # add the output layer to the array
    layers_dims.append(n_output)

    # number of layers
    L = len(layers_dims)

    for i in range(L):
        
        # if i==0 take n_x features
        if i ==0:
            layer_prev = n_input
        else:
            layer_prev = layers_dims[i-1]


        # check type of initialization
        if type_init.lower() == "random":

            parameters["W" + str(i+1)] = np.random.randn(layers_dims[i],layer_prev) * 10
            parameters["b" + str(i+1)] = np.zeros((layers_dims[i],1))

        elif type_init.lower() == "zeros":

            parameters["W" + str(i+1)] = np.zeros((layers_dims[i],layer_prev))
            parameters["b" + str(i+1)] = np.zeros((layers_dims[i],1))
        
        elif type_init.lower() == "he":

            parameters["W" + str(i+1)] = np.random.randn(layers_dims[i],layer_prev) * np.sqrt(2/layer_prev)
            parameters["b" + str(i+1)] = np.zeros((layers_dims[i],1))

        else:
            # default init
            parameters["W" + str(i+1)] = np.random.randn(layers_dims[i],layer_prev) * np.sqrt(2/layer_prev)
            parameters["b" + str(i+1)] = np.zeros((layers_dims[i],1))


    return parameters

In [21]:
# test initialization
hidden_layers_dim = [5,5,3]
n_input = 2
n_output = 1

params = initialize_parameters(hidden_layers_dim,n_input,n_output,"random")
params

{'W1': array([[ -2.23196235, -20.22602551],
        [  1.81994549,  15.46340317],
        [-13.51384752, -14.10711778],
        [ 11.35775439,  -8.62300619],
        [ -0.47108845,  -4.59912682]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[ -9.63900444,  21.66278564, -13.82293966, -11.24426518,
          10.93910559],
        [  7.15195874,  -1.67388275,  13.2481819 ,  15.04274093,
          15.6117558 ],
        [ -1.10045474,  -5.98659177,   0.04204417,   4.32545611,
          -2.81086023],
        [ -1.0767621 ,  -8.00222512,  -9.87145748,  15.63691978,
           0.55910288],
        [  1.20165507,  11.22528053,  -7.31075474, -10.28828527,
           9.57903627]]),
 'b2': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 'W3': array([[ -9.72090688,   8.74329264, -15.49729006,  12.50496593,
           1.13914404],
        [-13.81071519,  -3.44133073,  13.50196766,   9.62583621,
         -13.55031113],
        [

## 1.2 Forward propagation <a class="anchor" id="section_1_2"></a>

In [33]:
def activation_function(Z,activation_name):

    if activation_name.lower() == "sigmoid":

        A = 1/(1+np.exp(-Z))
    
    elif activation_name.lower() == "relu":

        A = np.maximum(0,Z)

    elif activation_name.lower() == "tanh":

        A = np.tanh(Z)
    
    else:
        # By default relu
        A = np.maximum(0,Z)

    return A

In [37]:
def forward_propagation(X,parameters,activation_name="relu"):

    """
    Compute the activation function
    
    Arguments:
    activation_name -- name of the activation function choosen
    Z -- items

    Returns:
    activation -- activation value
    """
    # init cache
    caches = []
    cache_layer = {}

    # layer
    L = len(parameters)//2

    # setting A_prev to X
    A_prev = X

    for i in range(1,L+1):

        # getting parameters
        W = parameters["W" + str(i)]
        b= parameters["b" + str(i)]

        # linear result
        Z = np.dot(W,A_prev) + b

        if i==L:
            # last layer -  sigmoid 
            A = activation_function(Z,"sigmoid")
        else:
            A = activation_function(Z,"relu")

        # adding to the cache
        cache = {f"W" : W, f"b":b,"A":A,"A_prev": A_prev}

        # adding layer cache
        caches.append(cache)

        # setting A_prev
        A_prev = A

    return A, caches

In [36]:
# test forward propagation

layers_dim = [5,5,3]
X = np.random.randn(2,100) *0.01
params = initialize_parameters(layers_dim,X.shape[0],1)

AL,caches = forward_propagation(X,params)

# check the shape of W 
for val in caches:
    print("W shape:",val["W"].shape,"b shape:",val["b"].shape)

W shape: (5, 2) b shape: (5, 1)
W shape: (5, 5) b shape: (5, 1)
W shape: (3, 5) b shape: (3, 1)
W shape: (1, 3) b shape: (1, 1)


## 1.3 Compute Loss <a class="anchor" id="section_1_3"></a>

## 1.4 Backward propagation <a class="anchor" id="section_1_4"></a>

## 1.5 Update parameters <a class="anchor" id="section_1_5"></a>

## 1.6 Predict <a class="anchor" id="section_1_6"></a>

## 1.7 Model <a class="anchor" id="section_1_7"></a>

# 2. Example on dataset 1 <a class="anchor" id="chapter2"></a>

# 3. Example on dataset 2 <a class="anchor" id="chapter3"></a>